# QUEENS: API example usage

This notebook explains how to make requests to the QUEENS API. Before running the code below, make sure you have installed `queens` and you have ingested, staged and served some data. Running the code below in a terminal should be enough to have this workbook running.

```
queens ingest dukes --table 6.1
queens stage dukes
queens serve --host 127.0.0.1 --port 8000
```

Once the app is up and running, you can navigate to the Swagger UI (in this case `http://127.0.0.1:8000/docs` to see the user interface and view the log of all requests you make (as well as the API documentation).

The QUEENS API has two endpoints: `data/{collection}` to request actual data and `metadata/{collection}` to request table metadata. These function in a similar way as the librayr methods `.query()` and `.metadata()` respectively.

To avoid heavy requests, the QUEENS API implements a row limit (default 1,000, extensible to 5,000 max) and pagination through a cursor. Each response gives the next cursor value so that users can request different pages. See below for a simple way to request the whole table.


In [ ]:
# get metadata (which columns are queryable)
import requests
import pandas as pd
import json

BASE = "http://127.0.0.1:8000"

# /metadata/{collection}?table_name=...
resp = requests.get(f"{BASE}/metadata/dukes", params={"table_name": "6.1"})
resp.raise_for_status()

payload = resp.json()

meta = pd.DataFrame(payload["data"])
meta


In [ ]:
# request some data

import json
import requests
import pandas as pd

BASE = "http://127.0.0.1:8000"

filters = {
    "year": {"gte": 2015},
    "fuel": {"like": "%wood%"}  # matches 'Wood', 'Waste wood'
}

# alternative with or
filters = {
    "year": {"gte": 2015},
    "$or": [
        {"fuel": {"eq": "Landfill gas"}},
        {"fuel": {"eq": "Waste wood"}}
    ]
}

params = {
    "table_name": "6.1",
    "filters": json.dumps(filters),
    "limit": 1000,   # optional (default on server), max is 5000
    # "cursor": None # optional for pagination; omit for first page
}

resp = requests.get(f"{BASE}/data/dukes", params=params)
resp.raise_for_status()

payload = resp.json()
df = pd.DataFrame(payload["data"])
print("Next cursor:", payload["next_cursor"])
df.head()


In [ ]:
# if you want to request the whole table (beyond the max row limit of 5_000 rows)

import json
import requests
import pandas as pd

BASE = "http://127.0.0.1:8000"

def fetch_all_6_1(filters=None, page_limit=5000):
    """Pull all rows from /data/dukes for table 6.1, respecting cursor pagination."""
    filters = filters or {}
    cursor = None
    all_rows = []

    while True:
        params = {
            "table_name": "6.1",
            "limit": page_limit,
        }
        if filters:
            params["filters"] = json.dumps(filters)
        if cursor is not None:
            params["cursor"] = cursor

        resp = requests.get(f"{BASE}/data/dukes", params=params)
        resp.raise_for_status()
        payload = resp.json()

        all_rows.extend(payload["data"])
        cursor = payload["next_cursor"]

        # cursor will ne None if there are no more pages
        if cursor is None:
            break

    return pd.DataFrame(all_rows)

# a) whole table
df_all = fetch_all_6_1()

# b) or: whole table after 2010 only
df_after_2010 = fetch_all_6_1(filters={"year": {"gt": 2010}})

len(df_all), df_all.head()
